In [1]:
import pandas as pd, json, pymysql, re, ast
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots
import plotly.graph_objects as go


pio.renderers.default = 'iframe'

In [2]:
# Create a function for easily querying data.
def qry(sql):

    # Open database connection
    connection = pymysql.connect(host="34.133.40.159",user="root",password="rootroot",database="project" )
    df = pd.read_sql(sql, connection)

    # disconnect from server
    connection.close()
    
    # return data.
    return df

#### Create visualization for a brand's (BENEFIT cosmetics) customer's statistics per metric (eye color, skin tone, and age)

In [3]:
def getMetric(metric):
    c = {}
    df = qry("""
        SELECT p.name, r.context_data_values FROM brands b, products p, reviews r
        WHERE b.brand_index = p.brand AND p.product_id = r.product_id
        AND b.brand_name LIKE 'benefit%'""")
    for k, v in df.iterrows():
        value = ast.literal_eval(v['context_data_values'])
        if metric in value.keys():
            response = value[metric]['Value']
            if response in c.keys():
                c[response] += 1
            else:
                c[response] = 1
    result=pd.DataFrame(c, index=['count']).T.reset_index()
    return (px.bar(result.sort_values('index'), x = 'count', y ='index', color=result['index'].to_list(),
                  text='count', title = f"BENEFIT customer {metric}"), result)



In [116]:
fig = make_subplots(rows=1, cols=3, 
                    subplot_titles=("BENEFIT customer ages", "BENEFIT customer skin tones", "BENEFIT customer eye colors")
                    )
df = getMetric('age')[1].sort_values('index')
fig.add_trace(
    go.Bar(x=df['index'], y=df['count'], name='count'),
    row=1, col=1
)
df = getMetric('skinTone')[1].sort_values('index')
fig.add_trace(
    go.Bar(x=df['index'], y=df['count'], name='count'),
    row=1, col=2
)
df = getMetric('eyeColor')[1].sort_values('index')
fig.add_trace(
    go.Bar(x=df['index'], y=df['count'], name='count'),
    row=1, col=3
)
fig.update_layout(title_text = "BENEFIT's Customerbase Demographic and Attribute Distribution", showlegend=False)

fig.show()

#### Create visualization for a moving and cumulative average time series for a specific product

In [121]:
pid = 'P429903' #P7109
df = qry(f"""SELECT r.*, p.name, b.brand_name
FROM reviews r, products p, brands b
WHERE r.product_id = p.product_id AND b.brand_index = p.brand
AND p.product_id = '{pid}' """)

In [122]:
df['submission_time'] = pd.to_datetime(df['submission_time'])
df = df.sort_values('submission_time')

In [129]:
name = df['name'][0]
brand = df['brand_name'][0]
period = 14
df[ 'rolling_avg' ] = df['rating'].rolling(period).mean()
fig = go.Figure([go.Scatter(x=df['submission_time'], y=df['rolling_avg'])])
fig.update_layout(title = f'{period}-day Moving Average Rating for:<br><b>{brand}</b>\'s "{name}"')

In [132]:
df[ 'cum_avg' ] = df['rating'].expanding(period).mean()
fig = go.Figure([go.Scatter(x=df['submission_time'], y=df['cum_avg'])])
fig.update_layout(title = f'{period}-day Cumulative Average Rating for:<br><b>{brand}</b>\'s "{name}"')

In [12]:
import pymongo
from pymongo import MongoClient

### Applying MongoDB for visualization
#### Find the most common use case for a specific product

In [133]:
client = MongoClient('mongodb://localhost:27017/project')
db=client.project

In [134]:
mongodf = pd.DataFrame(list(db.reviews_mongo.find({ })))

In [135]:
mongodf = mongodf.drop('_id', axis = 1)

In [136]:
brand = 'philosophy'
name = 'Purity Made Simple Cleanser'
px.histogram(mongodf.query('product_id == "P7109"')['skinConcerns'].dropna().sort_values(ascending=True),
       title = f'Main concerns for getting <br><b>{brand}</b>\'s "{name}')

### Connecting MongoDB, pandas and MySQL
#### Top products for specific skin type, skin tone and eye color
First, get the products with the right user attibute. Using the product id, find the product details on SQL database.
Finally, plot a graph showing the products rating and "love" statistic

In [142]:
mongodf2 = pd.DataFrame(list(db.reviews_mongo.find({"$and" : [{ "skinType": "combination" }, {"skinTone" : "light"}, {"eyeColor" : "blue" }]})))

In [143]:
plist = mongodf2.groupby('product_id').count()['_id'].sort_values(ascending = False).index.to_list()

In [152]:
topdf = qry(f"""SELECT CONCAT(b.brand_name, " ", name) as name ,rating, love 
FROM products p, brands b 
WHERE b.brand_index = p.brand AND category like "foundation" AND product_id IN('{"', '".join([str(x) for x in plist])}')""").head(5)

In [153]:
px.bar(topdf, x = 'love', y='name', text='rating')